In [8]:
from mlflow import MlflowClient
from mlflow.entities import ViewType
import mlflow
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [9]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [10]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1717423860705, experiment_id='2', last_update_time=1717423860705, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1717124023425, experiment_id='1', last_update_time=1717124023425, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1717123429709, experiment_id='0', last_update_time=1717123429709, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
client.create_experiment("my-cool-experiment")

In [12]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [14]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse {run.data.metrics['rmse']:.4f}")

run id: 727cc77259a24c8cb36f09b56a65a035, rmse 6.3184
run id: 212b8d9c7a5642e78f2fb8335e91fbcc, rmse 6.3184
run id: 38564173cbf84d63a7fced9c181b8f1e, rmse 6.3184
run id: e60f41265a88440cac11034f43e39308, rmse 6.3233
run id: 12f616d25d4b4f489e9072b74732ff9e, rmse 6.3235


In [15]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [16]:
run_id = "727cc77259a24c8cb36f09b56a65a035"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Successfully registered model 'nyc-taxi-regressor'.
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1717424876961, current_stage='None', description=None, last_updated_timestamp=1717424876961, name='nyc-taxi-regressor', run_id='727cc77259a24c8cb36f09b56a65a035', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/727cc77259a24c8cb36f09b56a65a035/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [18]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_5852/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [28]:
model_version = 2
new_stage = "Staging"
client.

/tmp/ipykernel_5852/4090915774.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717425487027, current_stage='Staging', description='', last_updated_timestamp=1717427559174, name='nyc-taxi-regressor', run_id='727cc77259a24c8cb36f09b56a65a035', run_link='', source='models:/nyc-taxi-regressor/1', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [29]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1717425487027, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-06-03', last_updated_timestamp=1717427740933, name='nyc-taxi-regressor', run_id='727cc77259a24c8cb36f09b56a65a035', run_link='', source='models:/nyc-taxi-regressor/1', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [ ]:
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    df_copy = df
    df_copy[categorical] = df[categorical].astype(str)
    df = df_copy

    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/nyc-taxi-regressor/versions/3")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [34]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")


In [35]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [36]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [37]:
X_test = preprocess(df, dv)


In [38]:
target = "duration"
y_test = df[target].values

In [41]:
model_name

'nyc-taxi-regressor'

In [43]:
%time test_model(name=model_name, stage="models_mlflow", X_test=X_test, y_test=y_test)


MlflowException: Invalid Model Version stage: models_mlflow. Value must be one of None, Staging, Production, Archived.